In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neurips-adc-challenge-2024-calibrated-tfrecord/primary-neurips-dataset-20241015205628-c1d1f1m1b30.tfrecords


In [2]:
primary_dataset = tf.data.TFRecordDataset(
    "/kaggle/input/neurips-adc-challenge-2024-calibrated-tfrecord/primary-neurips-dataset-20241015205628-c1d1f1m1b30.tfrecords")

## Decode`tf.train.Examples` in the `TFRecord`s

In [3]:
def decode_fn(proto):
    features_formats = {
        "id": tf.io.FixedLenFeature([], tf.int64),
        "airs": tf.io.FixedLenFeature([], tf.string),
        "fgs": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(proto, features_formats)
    pid = example["id"]
    airs = tf.io.parse_tensor(example["airs"], tf.float64)
    fgs = tf.io.parse_tensor(example["fgs"], tf.float64)
    target = tf.io.parse_tensor(example["target"], tf.float64)
    return pid, airs, fgs, target

In [4]:
def concatenate_signals(proto):
    _, airs, fgs, target = decode_fn(proto)
    signal = np.concatenate(
        [airs.numpy(), fgs.numpy().sum(axis=2)[:,:, np.newaxis,:]], axis=2
    )
    signal = tf.convert_to_tensor(signal)
    return signal, target

In [5]:
dataset = primary_dataset.prefetch(
    buffer_size=tf.data.AUTOTUNE).map(lambda example: tf.py_function(
    func=concatenate_signals, inp=[example],
    Tout=[
        tf.TensorSpec(shape=None, dtype=tf.float64),
        tf.TensorSpec(shape=None, dtype=tf.float64)
    ]))

In [104]:
# x, y = batch
# x = x.numpy().reshape(-1, 187, 283, 32)
# x = x.sum(axis=3).sum(axis=2) / (x.sum(axis=3).mean(axis=1).mean(axis=1)[: , np.newaxis] / 30)  #  /30 because of binning

In [12]:
# img = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
# planet = np.array([img + 1, img + 2])
# ds = np.array([planet + 1, planet + 2])

In [6]:
np_signal_dataset = None
np_target_dataset = None
for obs in tqdm(iter(dataset)):
    try:
        if np_signal_dataset is None:
            np_signal_dataset = obs[0].numpy()
            np_target_dataset = obs[1].numpy()
        else:
            np_signal_dataset = np.concatenate([np_signal_dataset, obs[0].numpy()])
            np_target_dataset = np.concatenate([np_target_dataset, obs[1].numpy()])
    except ValueError:
        pass

0it [00:00, ?it/s]

In [7]:
def make_example(x: tf.Tensor, y: tf.Tensor):
    x_ft = tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[
                tf.io.serialize_tensor(x).numpy(),
            ]
        )
    )
    y_ft = tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[
                tf.io.serialize_tensor(y).numpy(),
            ]
        )
    )

    features = tf.train.Features(
        feature={"x": x_ft, "y": y_ft}
    )
    example = tf.train.Example(features=features)
    return example.SerializeToString()

In [8]:
def save_to_tfrecords(dataset: tf.data.Dataset, path: str) -> None:
    """Save a dataset to TFRecord.

    Args:
        dataset (tf.data.Dataset): A Tensorflow Dataset
        path (str): Path to save the dataset
    """
    with tf.io.TFRecordWriter(path) as file_writer:
        for record in dataset:
            try:
                file_writer.write(record.numpy())
            except AttributeError:
                file_writer.write(record)

In [9]:
train_dataset = tf.data.Dataset.from_tensors((np_signal_dataset, np_target_dataset))
train_dataset = train_dataset.map(lambda x, y: tf.py_function(
    func=make_example, inp=[x, y],
    Tout=tf.string))


In [ ]:
save_to_tfrecords(train_dataset, "/kaggle/working/primary-neurips-dataset-20241015205628-c1d1f1m1b30-train.tfrecords")

Dans la suite on calcule un bruit blanc: La somme de tous les signaux par image que l'on divise par un signale blanc moyen par planete

In [ ]:
x = np_signal_dataset.sum(axis=3).sum(axis=2) / (np_signal_dataset.sum(axis=3).mean(axis=1).mean(axis=1)[: , np.newaxis] / 30)
y = np_target_dataset.mean(axis=1)
del np_signal_dataset, np_target_dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, shuffle=True, random_state=7)
scaler_xt = MinMaxScaler()
scaler_xte = MinMaxScaler()
scaler_yt = MinMaxScaler()
scaler_yte = MinMaxScaler()

x_train_scalled = scaler_xt.fit_transform(x_train)
x_test_scalled = scaler_xte.fit_transform(x_test)
y_train_scalled = scaler_yt.fit_transform(y_train)
y_test_scalled = scaler_yte.fit_transform(y_test)